In [1]:
import tensorflow  as tf
import tensorflow_datasets as tfds
import numpy as np  
import matplotlib.pyplot as plt
from PIL import Image
import cv2 as cv
from tensorflow import keras

In [2]:
import tensorflow as tf 

if tf.test.gpu_device_name(): 

        print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
        ess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))
        print('\n',ess)
else:

        print("Please install GPU version of TF")

Default GPU Device: /device:GPU:0
Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: NVIDIA GeForce GTX 1650 with Max-Q Design, pci bus id: 0000:01:00.0, compute capability: 7.5




In [3]:
(train_image,val_image),metadata = tfds.load('tf_flowers',
                                                 split=['train[:80%]','train[20%:80%]'],
                                                 data_dir="./dataset",
                                                 as_supervised=True,
                                                 with_info=True)

In [4]:
names_flowers = metadata.features['label'].names
names_flowers

['dandelion', 'daisy', 'tulips', 'sunflowers', 'roses']

# Data Augmentation

In [ ]:
img_pix = 224
data_augmentation = keras.Sequential(
  [
    #keras.layers.RandomFlip("horizontal_and_vertical"),
    keras.layers.RandomRotation(0.1),
    #keras.layers.RandomContrast(0.2),
    keras.layers.RandomZoom(height_factor=(-0.05, -0.15),width_factor=(-0.05, -0.15)),
  ]
)
    

In [ ]:

def format_image(image,label):
    image = tf.image.resize(image, (img_pix, img_pix))/255.0
    return image, label
def format_image_val(image,label):
    image = tf.image.resize(image, (img_pix, img_pix))/255.0
    return image, label   

In [ ]:
def aug_image(image,label):
    image_batch=image
    return image_batch

# Autotuning

In [ ]:
AUTOTUNE=tf.data.AUTOTUNE
BATCH_SIZE = 70

train_batches = train_image.map(format_image)
train_batches = train_batches.map(lambda x, y:(data_augmentation(x ), y),num_parallel_calls=AUTOTUNE)
train_batches=train_batches.batch(BATCH_SIZE)


validation_batches = val_image.map(format_image_val).batch(BATCH_SIZE)

#test_batches=test_image.map(format_image_val).batch(BATCH_SIZE)



In [ ]:

train_batches = train_batches.prefetch(buffer_size=AUTOTUNE)
validation_batches   = validation_batches.prefetch(buffer_size=AUTOTUNE)
#test_batches=test_batches.prefetch(buffer_size=AUTOTUNE)

# Callbacks Function

In [ ]:
class mycallbacks(keras.callbacks.Callback):
    def on_epoch_end(self,epoch,logs={}):
        self.epoch=epoch
        
        if(logs.get('accuracy')>0.95):
            print("\nReached 96% of accuracy, so cancelling training! ")
            self.model.stop_training=True
    def epoch_range(self):
        return self.epoch

In [ ]:
callbacks=mycallbacks()

model=keras.models.Sequential([
    
    keras.layers.Conv2D(16,(3,3),activation='relu',padding='valid',input_shape=(img_pix,img_pix,3)),
    keras.layers.AveragePooling2D(),
    keras.layers.Conv2D(32,(3,3),activation='relu'),
    keras.layers.MaxPool2D(),
    keras.layers.Conv2D(64,(3,3),activation='relu'),
    keras.layers.MaxPool2D(),
    keras.layers.Conv2D(96,(3,3),activation='relu'),
    keras.layers.MaxPool2D(),
    keras.layers.Dropout(0.3),
    keras.layers.Flatten(),
    keras.layers.Dense(1024,activation='relu'),
    keras.layers.Dense(512,activation='relu'),
    keras.layers.Dense(128,activation='relu'),
    keras.layers.Dense(5,activation='softmax')
])
model.compile(optimizer='adam',loss=keras.losses.SparseCategoricalCrossentropy(),metrics=['accuracy'])
accandlos=model.fit(train_batches,epochs=55, validation_data=validation_batches,callbacks=[callbacks])

In [ ]:
#model.evaluate(test_batches)

In [ ]:
accuracy_train=accandlos.history['accuracy']
accuracy_validate=accandlos.history['val_accuracy']

loss_train=accandlos.history['loss']
loss_validate=accandlos.history['val_loss']


In [ ]:
epochs_range=callbacks.epoch_range()
print(epochs_range)
epochs_range=range(epochs_range+1)

## Plotting Validation curve

In [ ]:
plt.figure(figsize=(15,4))
plt.subplot(1,2,1)
plt.plot(epochs_range,accuracy_train,label='TRAINING_ACCURACY')
plt.plot(epochs_range,accuracy_validate,label='VALIDATION_ACCURACY')
plt.legend(loc='lower right')
plt.title("Traning and Validation Accuracy")

plt.subplot(1,2,2)
plt.plot(epochs_range,loss_train,label='TRAINING_LOSS')
plt.plot(epochs_range,loss_validate,label='VALIDATION_LOSS')
plt.legend(loc='upper right')
plt.title("Traning and Validation Loss")
plt.show()

In [ ]:
model.summary()

In [ ]:
#!mkdir -p saved_model2
model.save('saved_model2/my_model15_-2/12/21')#model saved

In [ ]:
predict_path='1200px-DandelionFlower.jpg'
img= keras.utils.load_img( predict_path, target_size=(img_pix, img_pix))

In [ ]:
img

In [ ]:
img_arr = keras.utils.img_to_array(img)
img_arr = tf.expand_dims(img_arr, 0)

In [ ]:
img_arr

In [ ]:
predictions_neuron = model.predict(img_arr)

In [ ]:
print(predictions_neuron[0])

In [ ]:
prediction_score=tf.nn.softmax(predictions_neuron)
print(np.max(prediction_score))

In [ ]:
name_index=np.argmax(predictions_neuron[0])

In [ ]:
print("The flower classified is {}".format(names_flowers[name_index]))